## 3. Kollaboratives Filtern (Collaborated Filtering : basiert auf Nutzerbewertung)

Ich : Up, Zoomania  
Andere : Up, Zoomannia, Alles steht Kopf

Wenn jemand, der die gleichen Filme wie ich gesehen hat, "Alles steht Kopf" auch für gut bewertet, ist es wahrscheinlich, dass mir der Film auch gefallen wird.  
Im Gegensatz zum inhaltsbasierten Filtern braucht man hier keine Informationen über die Filme selbst. Es geht nur um die Bewertungshistorie der Filme.

Um ein Empfehlungssystem zu implementieren, muss man das Package "surprise" installieren > in der Anaconda-Prompt (als Administrator ausführen)
> pip install scikit-surprise

pip install scikit-surprise Fehler?
> Auf Jupyte Notebook : !pip install scikit-surprise / !pip install surprise

immer noch Fehler?
> anaconda cmd -> conda update --all -> conda install -c conda-forge scikit-surprise

In [1]:
import surprise
surprise.__version__

'1.1.3'

Bisher brauchten wir Informationen darüber, welcher Nutzer welche Filme wie bewertet haben > Kaggle  
https://surpriselib.com/

> Bei den Bewertungskennzahlen hat SVD++ mit einem RMS (Root Mean Square) von 0.92 den besten Wert, aber die Laufzeit ist sehr lang.  
> Wenn man Zeit hat, ist das kein Problem, aber wenn nicht, kann man auch ein Modell mit etwas schlechterer Leistung, aber kürzerer Laufzeit verwenden.

In [2]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [3]:
ratings = pd.read_csv('ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
ratings['rating'].min()

0.5

In [5]:
ratings['rating'].max()

5.0

Die Daten oben müssen nun in das vom "surprise-Paket" (Bibliothek) gewünschte Format gebracht werden. Dabei gibt es den Parameter "rating_scale" in der "Reader"-Klasse:  
> rating_scale (tuple, optional) – Die Bewertungsskala, die für jede Bewertung verwendet wird. Der Standardwert ist (1, 5)

Da unsere Daten eine Skala von (0.5, 5) haben, müssen wir das anpassen.

In [6]:
reader = Reader(rating_scale = (0.5, 5))

In [7]:
# Dabei muss "df" nur die Spalten "userId", "itemId", "rating" enthalten (Bibliotheksvorgabe)
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader = reader)
data

In [8]:
svd = SVD(random_state = 0)

Auf Kaggle gibt es eine Funktion "evaluate", um Daten über Modell (svd) zu bewerten, aber die ist veraltet (deprecated), also nicht mehr unterstützt. Stattdessen kann man "cross_validate"-Funktion verwenden.

In [9]:
cross_validate(svd, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True) # Root Mean Squared Error, Mean Absolute Error"

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8939  0.8908  0.9028  0.8835  0.9021  0.8946  0.0072  
MAE (testset)     0.6895  0.6875  0.6932  0.6824  0.6926  0.6890  0.0039  
Fit time          1.92    1.50    2.84    1.38    2.58    2.04    0.58    
Test time         0.19    0.21    0.26    0.24    0.19    0.22    0.03    


{'test_rmse': array([0.893926  , 0.89081194, 0.90283861, 0.88352566, 0.90211701]),
 'test_mae': array([0.68945095, 0.68754668, 0.69324892, 0.68238576, 0.69255394]),
 'fit_time': (1.9230287075042725,
  1.499971866607666,
  2.841823101043701,
  1.3757073879241943,
  2.576093912124634),
 'test_time': (0.18567204475402832,
  0.20958185195922852,
  0.26209473609924316,
  0.2401564121246338,
  0.18981552124023438)}

cross_validate : Beim Modelltraining werden die Daten in mehrere Testsets aufgeteilt und kreuzweise evaluiert.  
> Beispiel : 100 Datenpunkte  
> cv = 5 : die Daten in 5 Testsets aufteilen und evaluieren  
> z.B., A : 1-20, B: 21-40, C: 41-60, D: 61-80, E: 81-100  
> - ABCD (train set), E (test set)  
> - ABCE (train set), D (test set)  
> - ABDE (train set), C (test set)  
> - ACDE (train set), B (test set)  
> - BCDE (train set), A (test set)
> 
> So werden die Testläufe durchgewechselt und das Durchschnittsergebnis als Leistungsmaß verwendet.  
> Dies ist das "k-fold cross_validate" (auch in "scikit-learn-Bibliothek möglich))  
> Es gibt noch andere Methoden (z.B. Train-/Testset-Aufteilungsverhältnis wie vorher)

In [10]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [11]:
ratings[ratings['userId'] == 1]
# Daten des Nutzers mit "userId == 1" aus dem "ratings"-DataFrame

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


Vorhersage, wie der Nutzer mit "userID == 1" den Film mit "movieId == 302" bewerten würde:  
> svd.predict(userId, movieId, [r_ui])

In [12]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.7142061734434044, details={'was_impossible': False})

Ergebnisinterpretation :  
Prediction(uid=1, iid=302, r_ui=None, est=2.7142061734434044, details={'was_impossible': False})  
- uid = 1 (userId == 1)
- iid = 302 (movieId == 302, Bewertungsobjekt)
- r_ui = None (tatsächliche Bewertung, optional, keine Eingabe: None)
- est = 2.714 (vorhergesagte Bewertung)

In [13]:
svd.predict(1, 1029, 3)
# Wenn Nutzer 1 (userId == 1) den Film 1029 (movieId == 1029) mit 3 Punkten (r_ui = 3) bewertet, wie lautet die Vorhersage?
# Ergebnis: est = 2.88... so ähnlich wie seiner tatsächlicher Bewertung

Prediction(uid=1, iid=1029, r_ui=3, est=2.8814455446761933, details={'was_impossible': False})

In [14]:
ratings[ratings['userId'] == 100]
# Bewertungshistorie vom Nutzer 100

,userId,movieId,rating,timestamp
15273,100,1,4.0,854193977
15274,100,3,4.0,854194024
15275,100,6,3.0,854194023
15276,100,7,3.0,854194024
15277,100,25,4.0,854193977
15278,100,32,5.0,854193977
15279,100,52,3.0,854194056
15280,100,62,3.0,854193977
15281,100,86,3.0,854194208
15282,100,88,2.0,854194208


In [15]:
svd.predict(100, 1029) # userId == 100, movieId == 1029
# Ergebnis : est = 3.77..., das heißt, der Nutzer 100 würde 3.77 Punkt für diesen Film 1029 bewerten

Prediction(uid=100, iid=1029, r_ui=None, est=3.7705476478414846, details={'was_impossible': False})

Für den gleichen Film (movieId == 1029) ergeben sich je nach Nutzer unterschiedliche Vorhersagen, basierend auf den Bewertungshistorien der Nutzer
-> Kollaboratives Filtern

Man könnte die bereits gesehenen Filme und die noch nicht gesehenen Filme vom Nutzer separat erfassen, diese Daten zum Originaldatensatz \"ratings\" hinzufügen, das Modell trainieren und dann Vorhersagen für die noch nicht gesehenen Filme treffen. Die Top 10 der höchsten Vorhersagen wären dann die personalisierten Empfehlungen


Für eine Onlineshop wäre das sehr hilfreich. Mit den Kaufdaten und Bewertungen der Kunden könnte man mithilfe von \"surprise\" neue, passende Produktempfehlungen generieren und so den Umsatz steigern.